<a href="https://colab.research.google.com/github/saadmoumad/NLP-on-Morocco-s-King-Speech/blob/master/Char_based.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
'''from google.colab import drive
drive.mount('/content/gdrive')'''

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
!git clone https://github.com/saadmoumad/NLP-on-Morocco-s-King-Speech

fatal: destination path 'NLP-on-Morocco-s-King-Speech' already exists and is not an empty directory.


In [0]:
#Scraping Discours from maroc.ma 
!python3 NLP-on-Morocco-s-King-Speech/DiscourScraping.py
#Already pushed with git-repo

In [0]:
import tensorflow as tf
import numpy as np
import os
import time

In [0]:
data_dir = 'NLP-on-Morocco-s-King-Speech/Disc_Text'
files_name = os.listdir(data_dir)
files_name.sort()
text =''

In [0]:
for file_name in files_name[1:]: #exclu .DS_Store file
  path = os.path.join(data_dir, file_name)
  text += open(path, 'rb').read().decode(encoding='utf-8')

In [0]:
len(text)

2434887

In [0]:
# The unique characters in all Discours
vocab = sorted(set(text))

154 unique characters
['\n', '\x1c', ' ', '!', '"', '%', '&', "'", '(', ')']


In [0]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])

In [0]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

In [0]:
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [0]:
for i in char_dataset.take(5):
  print(i)

tf.Tensor(83, shape=(), dtype=int64)
tf.Tensor(46, shape=(), dtype=int64)
tf.Tensor(56, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(40, shape=(), dtype=int64)


In [0]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [0]:
for i in dataset.take(2):
  print(i)

(<tf.Tensor: shape=(100,), dtype=int64, numpy=
array([ 83,  46,  56,   2,  40,  56,  65,  60,  74,  75, 101,   2,  67,
        60,   2,  45,  70,  64,   2,  40,  70,  63,  56,  68,  68,  60,
        59,   2,  49,  36,  11,   2,  72,  76,  60,   2,  31,  64,  60,
        76,   2,  39, 149,  56,  74,  74,  64,  74,  75,  60,  11,   2,
        56,   2,  56,  59,  73,  60,  74,  74, 101,   2,  68,  60,  73,
        58,  73,  60,  59,  64,  83,   2,  15,  21,   2,  69,  70,  77,
        60,  68,  57,  73,  60,   2,  76,  69,   2,  59,  64,  74,  58,
        70,  76,  73,  74,   2,  97,   2,  67,  56])>, <tf.Tensor: shape=(100,), dtype=int64, numpy=
array([ 46,  56,   2,  40,  56,  65,  60,  74,  75, 101,   2,  67,  60,
         2,  45,  70,  64,   2,  40,  70,  63,  56,  68,  68,  60,  59,
         2,  49,  36,  11,   2,  72,  76,  60,   2,  31,  64,  60,  76,
         2,  39, 149,  56,  74,  74,  64,  74,  75,  60,  11,   2,  56,
         2,  56,  59,  73,  60,  74,  74, 101,   2,  68,  60

In [0]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [0]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           39424     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 154)           157850    
Total params: 4,135,578
Trainable params: 4,135,578
Non-trainable params: 0
_________________________________________________________________


In [0]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [0]:
model.compile(optimizer='adam', loss=loss)

In [0]:
#Checkpoints for model training
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [0]:
EPOCHS=10

In [0]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
376/376 [==============================] - 53s 140ms/step - loss: 2.1334
Epoch 2/10
376/376 [==============================] - 53s 140ms/step - loss: 1.2288
Epoch 3/10
376/376 [==============================] - 53s 140ms/step - loss: 1.0373
Epoch 4/10
376/376 [==============================] - 53s 140ms/step - loss: 0.9610
Epoch 5/10
376/376 [==============================] - 53s 140ms/step - loss: 0.9123
Epoch 6/10
376/376 [==============================] - 52s 140ms/step - loss: 0.8765
Epoch 7/10
376/376 [==============================] - 53s 140ms/step - loss: 0.8469
Epoch 8/10
376/376 [==============================] - 53s 140ms/step - loss: 0.8223
Epoch 9/10
376/376 [==============================] - 53s 140ms/step - loss: 0.8005
Epoch 10/10
376/376 [==============================] - 53s 141ms/step - loss: 0.7805


In [0]:
#Rebuild with diff batch size and restore the weights from the latest checkpoint.
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [0]:
def generate_text(model, start_string):
  num_generate = 1000
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []

  temperature = 1.0

  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [0]:
print(generate_text(model, start_string=u"SM le Roi "))

SM le Roi chroce de fierté, des institutions constitutionnelles. Nul doute, nous incite à agirment de l'attachement à l'aune décision procéduration de la sécurité alimentaire et généraliser l'action sous-l'état du Kowett, pour qu'ils assument. Cette implimantalisation des nobles objectifs onusiennes, la cristante disposent déployée capitale et accrue, nous attendons des générations, à combattre et s notre gestion tendant à se s et aux waroc aux conditions de soumettrement, l'expertise pent aux valeurs d'une réforme efficace dans la perspective des valeurs faveurs axé, de stimuler les pays et les principes et les solutions efficaces puisque nous pouvons économique et social. Nous nous rencontons à des institutions, et particulièrement, et de mettre en Âœuvre poser, à travers le nom de Nos provinces du Sud du logement, il trouverassieurs, à toute Ma connaissance, de soumettre à cette occasion, que nous assurons, un pôle de développement, tant sur le plandiai des cens qui lui confèrent la